# RAMP on predicting cyclist traffic in Paris

<div>
<table style="width:100%; background-color:transparent;">
  <tr style="background-color:transparent;">
    <td align="left"; style="background-color:transparent; width: 40%;">
        <a href="https://dataia.eu">
            <img src="https://github.com/ramp-kits/bike_counters_skore/raw/main/img/DATAIA-h.png" width="450px"/>
        </a>
    </td>
    <td align="right"; style="background-color:transparent; width: 40%;">
        <a href="https://mdls.fr/">
            <img src="https://github.com/ramp-kits/bike_counters_skore/raw/main/img/probabl_logo.png" width="350px"/>
        </a>
    </td>
  </tr>
 </table>
</div>

Authors: *Roman Yurchak (Symerio), Sylvain Combettes (Probabl), François Caud (DATAIA)*; also partially inspired by the air_passengers starting kit.


## Introduction

The dataset was collected with cyclist counters installed by Paris city council in multiple locations. It contains hourly information about cyclist traffic, as well as the following features,
 - counter name
 - counter site name
 - date
 - counter installation date
 - latitude and longitude
 
Available features are quite scarce. However, **we can also use any external data that can help us to predict the target variable.** 

# Loading the data with pandas

First, if not done already, download the data files with the `download_data.py` script:
```bash
!conda activate ramp-bike-skore
!python download_data.py
```
It will untar the archive and put the train and test file in the data folder.


Data is stored in [Parquet format](https://parquet.apache.org/), an efficient columnar data format. We can load the train set with pandas,

In [ ]:
import pandas as pd
from pathlib import Path

data = pd.read_parquet(Path("data") / "train.parquet").sort_values(by="date")

In [3]:
data.head()

,counter_id,counter_name,site_id,site_name,bike_count,date,counter_installation_date,counter_technical_id,latitude,longitude,log_bike_count
705677,100056332-104056332,Pont de Bercy SO-NE,100056332,Pont de Bercy,0.0,2020-09-01 01:00:00,2019-12-11,Y2H19070378,48.83848,2.37587,0.000000
333389,100047547-104047547,6 rue Julia Bartet NE-SO,100047547,6 rue Julia Bartet,4.0,2020-09-01 01:00:00,2018-11-28,Y2H18086323,48.82636,2.30303,1.609438
343292,100047547-103047547,6 rue Julia Bartet SO-NE,100047547,6 rue Julia Bartet,2.0,2020-09-01 01:00:00,2018-11-28,Y2H18086323,48.82636,2.30303,1.098612
805911,100057380-103057380,Totem Cours la Reine O-E,100057380,Totem Cours la Reine,0.0,2020-09-01 01:00:00,2020-02-11,YTH19111509,48.86462,2.31444,0.000000
353162,100047548-103047548,Face au 25 quai de l'Oise NE-SO,100047548,Face au 25 quai de l'Oise,2.0,2020-09-01 01:00:00,2018-11-28,Y2H18086324,48.89141,2.38482,1.098612


Now, let us perform some exploratory data analysis on this dataframe:

In [4]:
from skrub import TableReport

TableReport(data)

Processing column  11 / 11


,,,,,,,,,,,


Note that [skrub's TableReport](https://skrub-data.org/stable/reference/generated/skrub.TableReport.html) is interactive and that there are several tabs. For example, in the Stats tab, you can focus on the number of unique entries for each feature.

We have a 30 counting sites where sometimes multiple counters are installed per location.  Let's look at the most frequented stations,

In [ ]:
data.groupby(["site_name", "counter_name"], observed=True)[
    "bike_count"
].sum().sort_values(ascending=False).head(10).to_frame()

# Visualizing the data


Let's visualize the data, starting from the spatial distribution of counters on the map

In [ ]:
import folium

m = folium.Map(location=data[["latitude", "longitude"]].mean(axis=0), zoom_start=13)

for _, row in (
    data[["counter_name", "latitude", "longitude"]]
    .drop_duplicates("counter_name")
    .iterrows()
):
    folium.Marker(
        row[["latitude", "longitude"]].values.tolist(), popup=row["counter_name"]
    ).add_to(m)

m

Note that in this RAMP problem we consider only the 30 most frequented counting sites, to limit data size.


Next, we will look into the temporal distribution of the most frequented bike counter. If we plot it directly we will not see much because there are half a million data points,

In [ ]:
import matplotlib.pyplot as plt

mask = data["counter_name"] == "Totem 73 boulevard de Sébastopol S-N"

data[mask].plot(x="date", y="bike_count")
plt.show()

Instead we aggregate the data, for instance, by week to have a clearer overall picture,

In [ ]:
mask = data["counter_name"] == "Totem 73 boulevard de Sébastopol S-N"

data[mask].groupby(pd.Grouper(freq="1W", key="date"))[["bike_count"]].sum().plot()
plt.show()

While at the same time, we can zoom on a week in particular for a more short-term visualization,

In [ ]:
fig, ax = plt.subplots(figsize=(10, 4))

mask = (
    (data["counter_name"] == "Totem 73 boulevard de Sébastopol S-N")
    & (data["date"] > pd.to_datetime("2021/03/01"))
    & (data["date"] < pd.to_datetime("2021/03/08"))
)

data[mask].plot(x="date", y="bike_count", ax=ax)
plt.show()

The hourly pattern has a clear variation between work days and weekends (7 and 8 March 2021).

If we look at the distribution of the target variable it skewed and non normal, 

In [ ]:
import seaborn as sns

ax = sns.histplot(data, x="bike_count", kde=True, bins=50)

Least square loss would not be appropriate to model it since it is designed for normal error distributions. One way to precede would be to transform the variable with a logarithmic transformation,
```py
data['log_bike_count'] = np.log(1 + data['bike_count'])
```

In [ ]:
ax = sns.histplot(data, x="log_bike_count", kde=True, bins=50)

which has a more pronounced central mode, but is still non symmetric. In the following, **we use `log_bike_count` as the target variable** as otherwise `bike_count` ranges over 3 orders of magnitude and least square loss would be dominated by the few large values. 

## Feature extraction

To account for the temporal aspects of the data, we cannot input the `date` field directly into the model. Instead we extract the features on different time-scales from the `date` field, 

In [ ]:
def _encode_dates(X):
    X = X.copy()  # modify a copy of X
    # Encode the date information from the DateOfDeparture columns
    X.loc[:, "year"] = X["date"].dt.year
    X.loc[:, "month"] = X["date"].dt.month
    X.loc[:, "day"] = X["date"].dt.day
    X.loc[:, "weekday"] = X["date"].dt.weekday
    X.loc[:, "hour"] = X["date"].dt.hour

    # Finally we can drop the original columns from the dataframe
    return X.drop(columns=["date"])

In [ ]:
data["date"].head()

In [ ]:
_encode_dates(data[["date"]].head())

To use this function with scikit-learn estimators we wrap it with [FunctionTransformer](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.FunctionTransformer.html),

In [ ]:
from sklearn.preprocessing import FunctionTransformer

date_encoder = FunctionTransformer(_encode_dates, validate=False)
date_encoder.fit_transform(data[["date"]]).head()

Since it is unlikely that, for instance, that `hour` is linearly correlated with the target variable, we would need to additionally encode categorical features for linear models. This is classically done with [OneHotEncoder](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.OneHotEncoder.html), though other encoding strategies exist.

In [ ]:
from sklearn.preprocessing import OneHotEncoder

enc = OneHotEncoder(sparse_output=False)

enc.fit_transform(_encode_dates(data[["date"]])[["hour"]].head())

## Ridge model

Let's now construct our first linear model with [Ridge](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.Ridge.html). We use a few helper functions defined in `problem.py` of the starting kit to load the public train and test data:

In [ ]:
import problem

X_train, y_train = problem.get_train_data()
X_test, y_test = problem.get_test_data()

In [ ]:
X_train.head(2)

and

In [ ]:
y_train

Where `y` contains the `log_bike_count` variable. 

The test set is in the future as compared to the train set,

In [ ]:
print(
    f'Train: n_samples={X_train.shape[0]},  {X_train["date"].min()} to {X_train["date"].max()}'
)
print(
    f'Test: n_samples={X_test.shape[0]},  {X_test["date"].min()} to {X_test["date"].max()}'
)

In [ ]:
_encode_dates(X_train[["date"]]).columns.tolist()

In [ ]:
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import Ridge
from sklearn.pipeline import make_pipeline

date_encoder = FunctionTransformer(_encode_dates)
date_cols = _encode_dates(X_train[["date"]]).columns.tolist()

categorical_encoder = OneHotEncoder(handle_unknown="ignore")
categorical_cols = ["counter_name", "site_name"]

preprocessor = ColumnTransformer(
    [
        ("date", OneHotEncoder(handle_unknown="ignore"), date_cols),
        ("cat", categorical_encoder, categorical_cols),
    ]
)

regressor = Ridge()

pipe = make_pipeline(date_encoder, preprocessor, regressor)
pipe

### Evaluation

Let us use the estimator report from skore to evaluate our scikit-learn pipeline:

In [ ]:
from skore import EstimatorReport

pipe_report = EstimatorReport(
    pipe,
    X_train=X_train,
    X_test=X_test,
    y_train=y_train,
    y_test=y_test,
)

We display the helper to see what is available to us:

In [ ]:
pipe_report.help()

We evaluate our model on both the $R^2$ and the RMSE, using the metrics accessor:

In [ ]:
pipe_report.metrics.report_metrics(data_source="train")

In [ ]:
pipe_report.metrics.report_metrics(data_source="test")

The model doesn't have enough capacity to generalize on the train set, since we have lots of data with relatively few parameters. However it happened to work somewhat better on the test set. We can compare these results with the baseline predicting the mean value,

In [ ]:
from sklearn.dummy import DummyRegressor

dummy_regr = DummyRegressor(strategy="mean")
dummy_report = EstimatorReport(
    dummy_regr,
    X_train=X_train,
    X_test=X_test,
    y_train=y_train,
    y_test=y_test,
)
df_dummy = pd.concat([
    dummy_report.metrics.report_metrics(data_source="train"),
    dummy_report.metrics.report_metrics(data_source="test"),
], axis=1)
df_dummy.columns = ["DummyRegressor (train)", "DummyRegressor (test)"]
df_dummy

In [ ]:
from skore import ComparisonReport

reports_to_compare = {
    "Ridge (manual)": pipe_report,
    "Dummy (mean)": dummy_report,
}
comparator = ComparisonReport(reports=reports_to_compare)
comparator.metrics.report_metrics()

which illustrates that we are performing better than the baseline.

Let's visualize the predictions for one of the stations,

In [ ]:
import numpy as np

pipe = pipe_report.estimator_

mask = (
    (X_test["counter_name"] == "Totem 73 boulevard de Sébastopol S-N")
    & (X_test["date"] > pd.to_datetime("2021/09/01"))
    & (X_test["date"] < pd.to_datetime("2021/09/08"))
)

df_viz = X_test.loc[mask].copy()
df_viz["bike_count"] = np.exp(y_test[mask.values]) - 1
df_viz["bike_count (predicted)"] = np.exp(pipe.predict(X_test[mask])) - 1

In [ ]:
fig, ax = plt.subplots(figsize=(12, 4))

df_viz.plot(x="date", y="bike_count", ax=ax)
df_viz.plot(x="date", y="bike_count (predicted)", ax=ax, ls="--")
ax.set_title("Predictions with Ridge")
ax.set_ylabel("bike_count")

So we start to see the daily trend, and some of the week day differences are accounted for, however we still miss the details and the spikes in the evening are under-estimated.

A useful way to visualize the error is to plot the prediction error,

In [ ]:
pipe_report.metrics.prediction_error().plot(kind="actual_vs_predicted")

## Linear model from skrub

In [ ]:
from skrub import tabular_learner

tab_learner = tabular_learner(Ridge())

tab_report = EstimatorReport(
    tab_learner,
    X_train=X_train,
    X_test=X_test,
    y_train=y_train,
    y_test=y_test,
)
tab_report.estimator_

In [ ]:
reports_to_compare["Ridge (skrub baseline)"] = tab_report
comparator = ComparisonReport(reports=reports_to_compare)
comparator.metrics.report_metrics()

## Tree-based model

In [ ]:
hgbt_skrub = tabular_learner("regressor")

hbbt_report = EstimatorReport(
    hgbt_skrub,
    X_train=X_train,
    X_test=X_test,
    y_train=y_train,
    y_test=y_test,
)
hbbt_report.estimator_

In [ ]:
reports_to_compare["HGBT (skrub baseline)"] = hbbt_report
comparator = ComparisonReport(reports=reports_to_compare)
comparator.metrics.report_metrics()

## A note on cross-validation

It is recommended to use cross-validation for hyper-parameter tuning with [GridSearchCV](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html) or more reliable model evaluation with [cross_val_score](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.cross_val_score.html#sklearn.model_selection.cross_val_score). In this case, because we want the test data to always be in the future as compared to the train data, we can use [TimeSeriesSplit](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.TimeSeriesSplit.html),

<img src="https://i.stack.imgur.com/Q37Bn.png" />

The disadvantage, is that we can either have the training set size be different for each fold which is not ideal for hyper-parameter tuning (current figure), or have constant sized small training set which is also not ideal given the data periodicity. This explains that generally we will have worse cross-validation scores than test scores, 

In [ ]:
from sklearn.model_selection import TimeSeriesSplit
from skore import CrossValidationReport

cv = TimeSeriesSplit(n_splits=6)

cv_report = CrossValidationReport(
    pipe, X_train, y_train, cv_splitter=cv
)

In [ ]:
cv_report.help()

In [ ]:
cv_report.metrics.report_metrics(data_source="train")

In [ ]:
cv_report.metrics.report_metrics(data_source="test")

In [ ]:
cv_report.metrics.report_metrics(data_source="test", aggregate=None)